In [6]:
import sys
import sqlite3
from pathlib import Path
import os

project_root = Path().resolve().parent  
sys.path.append(str(project_root))


In [13]:
PDF_PATH = project_root / "data/raw/ah/AH_kassabon_2025-08-25 185700_1209.pdf"
DB_PATH = project_root / "data/cleaned/receipts.db"

In [8]:
# connect to database
conn = sqlite3.connect(DB_PATH)
cursor = conn.cursor()

In [14]:
if os.path.isfile(PDF_PATH):
    print("Valid: file exists")
else:
    print("Invalid: not a file or doesn’t exist")

Valid: file exists


In [15]:
import pdfplumber

# returns the full receipt in a single string
# each product is separated by a new line
full_text = ""

with pdfplumber.open(PDF_PATH) as pdf:
    for page in pdf.pages:
        text = page.extract_text()
        full_text += text

In [16]:
print(full_text)

ALBERT HEIJN 1209
Harderwijkerweg 18
8071 GA Nunspeet 0341-252014
AANTAL OMSCHRIJVING PRIJS BEDRAG
BONUSKAART xx5298
2 OET PIZZA 4,99 9,98 B
2 OETKER PIZZA 4,99 9,98 B
1 MAESTRO WIT 5,29 B
3 NORTON 7,99 23,97 B
4 AH DRINKYOGH 1,29 5,16
2 DZH YOGHURT 2,29 4,58
1 AH BURRATINE 2,99
1 HARIBO 1,99
2 AH OLIJVEN 1,59 3,18
2 MENTOS GUM 4,69 9,38 B
2 POFFERTJES 1,89 3,78 B
2 PAPRIKA 0,65 1,30 B
2 WALDK HEEL 2,39 4,78
26 SUBTOTAAL 86,36
BONUS ElMaestroenT -1,00
BONUS MENTOSKAUWGO -4,69
BONUS AHPAPRIKAROO -0,31
BONUS NORTONWK35 -6,00
BONUS ALLEDR.OETKE -9,98
BONUS AHPANK8,POFF -1,89
UW VOORDEEL 23,87
Waarvan
BONUS BOX 0,00
TOTAAL 62,49
SPAARACTIES:
18 eSPAARZEGELS
BETAALD MET:
PINNEN 62,49
POI: 50068808
KLANTTICKET Terminal 4FMH8C
Merchant 1315641 Periode 5237
Transactie 01243746 P 5 A 0 R 01 : 363BCW0I3YEILNSNHGSWMK8R
Z Debit Mastercard
(A0000000041010) DEBIT MASTERCARD
Kaart 537476xxxxxx6830 Kaartserienummer 1
BETALING Datum 25/08/2025 19:00
Autorisatiecode 001B85 Totaal 62,49 EUR
Contactless L

In [ ]:
import re

# define booleans to catch start of receipt and end of receipt
start_receipt = 0
end_receipt = 0

filename = "AH_kassabon_2025-08-23 094800_1209.pdf"

# loop over all lines in the receipt
for line in text.splitlines():

    if 'SUBTOTAAL' in line:
        end_receipt = 1  

    # only for the line items: extarct the different components
    if start_receipt == 1 and end_receipt == 0:
        match = re.match(r"(\d+)?\s*([A-Z ]+)?\s*([\d,]+)?\s*([\d,]+)?\s*([A-Z])?", line)
        if match:
            quantity, product, price_unit, price_total, bonuscode = match.groups()
            
            print(match.groups())

            # transform values to database types
            quantity = int(quantity)
            product = product.strip()
            price_unit = float(price_unit.replace(",", "."))
            price_total = quantity * price_unit
            discount_ind = "J" if bonuscode == "B" else "N"

            # Insert a row into a table
            cursor.execute(
                "INSERT INTO receiptTable (filename, product, quantity, price_unit, price_total, discount_ind) VALUES (?, ?, ?, ?, ?, ?)",
                (filename, product, quantity, price_unit, price_total, discount_ind)
                )


    if 'BONUSKAART' in line:
        start_receipt = 1

# Commit changes
conn.commit()

# Close connection
conn.close()


('2', 'TIJGER VOLK ', '1,99', '3,98', 'B')
('1', 'AH VVP HAGEL ', '3,49', None, None)
('1', 'VENZ XXL ', '3,99', None, None)
('2', 'COOLBEST ', '2,29', '4,58', 'B')
('2', 'DONUT DECO ', '3,95', '7,90', 'B')
